In [1]:
import altair as alt
import json
import pandas as pd
import numpy as np
import warnings

from pybench.chart import grouped_bar_chart
import pybench.themes
from pybench.utils import (
    benchmark_json_to_pandas,
    compute_speedup,
    filter_by_string_in_column,
    significant_round,
    split_params_list,
)

In [2]:
benchmark_json_to_pandas.__globals__['__file__']


'/projects/a9009/tdm5510/workshops/codex_042025/py312_cupy_gpu/lib/python3.12/site-packages/pybench/utils.py'

In [3]:
pd.__version__

'2.2.3'

In [6]:
# Read a pytest-benchmark json into a pandas DataFrame.
json_dir = '/projects/a9009/tdm5510/workshops/codex_042025/cupy_bench/results/'
df_h100 = benchmark_json_to_pandas(json_dir + 'benchmark_results_h100.json')
df_a100 = benchmark_json_to_pandas(json_dir + 'benchmark_results_a100.json')

In [7]:
df_h100

,group,name,fullname,param,params.shape,params.module,options.disable_gc,options.timer,options.min_rounds,options.max_time,...,stats.outliers,stats.ld15iqr,stats.hd15iqr,stats.ops,stats.total,stats.data,stats.iterations,params.nrows,params.data_path,params.data
0,None,test_FFT[shape0-numpy],pybench/benchmarks/benchmark_array.py::test_FF...,shape0-numpy,"[1000, 1000]",numpy,False,perf_counter,5,1.0,...,1;0,0.004156,0.005700,215.995749,0.023149,"[0.004662969149649143, 0.005699878558516502, 0...",1,NaN,NaN,NaN
1,None,test_FFT[shape0-cupy],pybench/benchmarks/benchmark_array.py::test_FF...,shape0-cupy,"[1000, 1000]",cupy,False,perf_counter,5,1.0,...,1;0,0.000027,0.000168,15657.143124,0.000319,"[6.0725025832653046e-05, 0.0001684408634901046...",1,NaN,NaN,NaN
2,None,test_FFT[shape1-numpy],pybench/benchmarks/benchmark_array.py::test_FF...,shape1-numpy,"[10000, 10000]",numpy,False,perf_counter,5,1.0,...,2;0,0.710967,0.712759,1.404430,3.560162,"[0.7109671579673886, 0.7124414416030049, 0.712...",1,NaN,NaN,NaN
3,None,test_FFT[shape1-cupy],pybench/benchmarks/benchmark_array.py::test_FF...,shape1-cupy,"[10000, 10000]",cupy,False,perf_counter,5,1.0,...,1;1,0.001246,0.001432,775.449171,0.006448,"[0.0012662103399634361, 0.0014320239424705505,...",1,NaN,NaN,NaN
4,None,test_FFT[shape2-numpy],pybench/benchmarks/benchmark_array.py::test_FF...,shape2-numpy,"[20000, 20000]",numpy,False,perf_counter,5,1.0,...,1;1,2.891941,2.902665,0.345431,14.474677,"[2.89194115716964, 2.902664856053889, 2.893623...",1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,None,test_ElasticNet[/projects/a9009/tdm5510/worksh...,pybench/benchmarks/benchmark_ml.py::test_Elast...,/projects/a9009/tdm5510/workshops/codex_042025...,"[1048576, 512]",cuml,False,perf_counter,5,1.0,...,1;1,0.002895,0.004714,305.705322,0.016356,"[0.004714042879641056, 0.0029317056760191917, ...",1,NaN,NaN,/projects/a9009/tdm5510/workshops/codex_042025...
102,None,test_ElasticNet[/projects/a9009/tdm5510/worksh...,pybench/benchmarks/benchmark_ml.py::test_Elast...,/projects/a9009/tdm5510/workshops/codex_042025...,"[2097152, 512]",sklearn,False,perf_counter,5,1.0,...,2;0,0.991392,0.992353,1.008249,4.959093,"[0.9913923209533095, 0.9923528218641877, 0.992...",1,NaN,NaN,/projects/a9009/tdm5510/workshops/codex_042025...
103,None,test_ElasticNet[/projects/a9009/tdm5510/worksh...,pybench/benchmarks/benchmark_ml.py::test_Elast...,/projects/a9009/tdm5510/workshops/codex_042025...,"[2097152, 512]",cuml,False,perf_counter,5,1.0,...,1;1,0.003275,0.004643,279.663702,0.017879,"[0.0033914782106876373, 0.003285815939307213, ...",1,NaN,NaN,/projects/a9009/tdm5510/workshops/codex_042025...
104,None,test_ElasticNet[/projects/a9009/tdm5510/worksh...,pybench/benchmarks/benchmark_ml.py::test_Elast...,/projects/a9009/tdm5510/workshops/codex_042025...,"[4194304, 512]",sklearn,False,perf_counter,5,1.0,...,1;0,0.193492,0.193643,5.167035,0.967673,"[0.19354225508868694, 0.19349578581750393, 0.1...",1,NaN,NaN,/projects/a9009/tdm5510/workshops/codex_042025...


In [ ]:
# Split list 'params.shape' into multiple columns.
# In this example the list is split in 'params.shape.0' and 'params.shape.1'.
df_h100 = split_params_list(df_h100, 'params.shape')
df_a100 = split_params_list(df_a100, 'params.shape')

In [ ]:
# Split only "fast" operations into `cupy_df`.
cupy_df_h100 = filter_by_string_in_column(df_h100, 'name', 'cupy')
cupy_df_a100 = filter_by_string_in_column(df_a100, 'name', 'cupy')

In [ ]:
# Split only "slow" operations into `numpy_df`.
numpy_df_h100 = filter_by_string_in_column(df_h100, 'name', 'numpy')
numpy_df_a100 = filter_by_string_in_column(df_a100, 'name', 'numpy')

In [ ]:
# Define the list of operations we want to plot.
operation_list = [
    'FFT',
    'Sum',
    'Standard_Deviation',
    'Elementwise',
    'Matrix_Multiplication',
    'Array_Slicing',
    'SVD',
    'Stencil'
]

In [ ]:
# Define the list of parameters we will use to match entries in "fast" and
# "slow" DataFrames.
param_list = ['params.shape.0', 'params.shape.1']

In [ ]:
# Define the metric we want to use to compute the speedup.
stats_param = 'stats.median'

In [ ]:
# Compute the speedup DataFrame.
speedups_df_h100 = compute_speedup(numpy_df_h100, cupy_df_h100, operation_list, param_list, stats_param)
speedups_df_a100 = compute_speedup(numpy_df_a100, cupy_df_a100, operation_list, param_list, stats_param)

In [ ]:
speedups_df_h100

In [ ]:
# Remove dots from column names, altair doesn't seem to work with them.
speedups_df_h100 = speedups_df_h100.rename(columns={'params.shape.0': 'shape0'})
speedups_df_h100 = speedups_df_h100.rename(columns={'params.shape.1': 'shape1'})
speedups_df_a100 = speedups_df_a100.rename(columns={'params.shape.0': 'shape0'})
speedups_df_a100 = speedups_df_a100.rename(columns={'params.shape.1': 'shape1'})

# Drop rows based on certain conditions, such as their shapes
speedups_df_h100.drop(speedups_df_h100[speedups_df_h100['shape0'] == 20000].index, inplace=True)
speedups_df_a100.drop(speedups_df_a100[speedups_df_a100['shape0'] == 20000].index, inplace=True)


# Give human readable names to shapes and store them in column 'size'
speedups_df_h100['size'] = speedups_df_h100['shape0'].apply(lambda row: '800MB' if row == 10000 else '8MB')
speedups_df_a100['size'] = speedups_df_a100['shape0'].apply(lambda row: '800MB' if row == 10000 else '8MB')


# Replace underscores with spaces in operation names for better printing
speedups_df_h100['name'] = speedups_df_h100['name'].apply(lambda n: n.replace('_', ' '))
speedups_df_a100['name'] = speedups_df_a100['name'].apply(lambda n: n.replace('_', ' '))

In [ ]:
speedups_df_a100

In [ ]:
# Enable theme "RAPIDS" from pybench.themes
alt.themes.enable("RAPIDS")

# Create altair chart from `speedups_df`
chart = grouped_bar_chart(
    speedups_df_a100,
    'size',
    'speedup',
    'name',
    ['800MB', '8MB'],
    y_title='GPU Speedup Over CPU',
    y_scale_type='symlog',
    y_tick_count=5,
    group_title='Operation on A100 GPU',
    bar_title_angle=-20,
    legend_title='Array Size (MB)',
    group_height=500,
    group_width=80)

In [ ]:
chart

In [ ]:
# Save chart to HTML file
# chart.save('plot_array_example.html')